# Project INF8225: tutorial
**Knockoff Nets: Stealing Functionality of Black-Box Models, CVPR '19** 

This part presents the efforts done to reproduce the results of the paper and the additional work by our team.

**Challenges faced:** Although the authors provide a Github repository with their code, the reproduction of the results tooks us a lot of time (weeks) since the attacks are against complex models (ResNet-38, VGG-16,..) and the datasets are huge. The Readme written does not provide enough information and requires a good knowledge and experience in deep learning programming.

The forked repository for the project is on this link: https://github.com/KacemKhaled/knockoffnets


# Environment Preparation
**Disclaimer:** if you are using Windows, you may not be able to prepare the environment, we tried that for a while and it only lead us to *Errors Land*! So save your time and use Linux. In our case, we used Ubuntu 16.04.

#### Before you start: Setting up the GPU
The process to install the CUDA driver on Linux is complicated and time consuming, but it is worth it to gain more time in the future instead of working on the CPU. 

We have followed this tutorial: https://docs.nvidia.com/cuda/cuda-installation-guide-linux/index.html


## Steps 


*   Fork the original repository from : https://github.com/tribhuvanesh/knockoffnets, our forked repository is on this link: https://github.com/KacemKhaled/knockoffnets
*   Clone it into your own machine: 

```
$ git clone https://github.com/KacemKhaled/knockoffnets.git
```
*   If you don't have Anaconda installed, it would be better to install it (https://conda.io/projects/conda/en/latest/user-guide/install/linux.html)
*   Go to the repository and execute 

```
$ conda env create -f environment.yml
```

*   This should install all the requirements and create a conda enivornment, to start working in the project you should activate the environment (named "knockoff") using this command:

```
$ conda activate knockoff
```


# Experiments

### Attack: Overview

Following the content of the Readme Page the commands/steps provided will guide you to:

1. Train victim models (or download pretrained ResNet-34 models)
2. Train knockoff models (or download pretrained ResNet-34 models)
  * Constructing transfer sets
  * Training knockoff models

The framework provided by the authors is rich with predefined commands to run most of the experiments and reproduce them. However, training complicated architectures need a lot of GPU time. So we decided to reproduce the steps once for only one victim with different architecture to assess the expandability of the attacks.

### 1. Training Victim model
We have chosen a Resnet-18 architecture instead of Resnet-34 to train the victim model on CUBS200 dataset.

```
# Format:
$ python -m "knockoff.victim.train" DS_NAME ARCH -d DEV_ID \
        -o models/victim/VIC_DIR -e EPOCHS --pretrained
# where DS_NAME = {cubs200, caltech256, ...}, ARCH = {resnet18, vgg16, densenet161, ...}
# if the machine contains multiple GPUs, DEV_ID specifies which GPU to use

# We run this command
$ python -m "knockoff.victim.train" CUBS200 resnet18 -d 0 \
        -o models/victim/cubs200-resnet18 -e 10 --log-interval 25 \
        --pretrained imagenet
```

### 2.1. Constructing transfer sets
The chosen dataset to construct the transfer set in the original code is ImageNet, we have attempted to download it from the official website but our request to the organization is still pending. So we decided to use a part of the OpenImages dataset (only 58GB which is around 160k images). 

Handling the OpenImages dataset was not included in the project code so we added the necessary modifications for the purpose.

Then we executed this command to query 80k images from the dataset to the victim model with a random policy.

```
# Format
$ python -m "knockoff.adversary.transfer" random models/victim/VIC_DIR \
        --out_dir models/adversary/ADV_DIR --budget BUDGET \
        --queryset QUERY_SET --batch_size 8 -d DEV_ID
# where QUERY_SET = {ImageNet1k , OpenImages...}

# We run this command
$ python3 -m "knockoff.adversary.transfer" random models/victim/cubs200-resnet18 \
        --out_dir models/adversary/cubs200-resnet18-random --budget 80000 \
        --queryset OpenImages --batch_size 8 -d 0
```

### 2.2. Training the Knockoff model
The obtained transfer set will serve as a dataset to train a knockoff model, here we decided to train the model on an architecture different that the victim which had ResNet-18, so we chose ResNet-34.

We tried to train on different architectures like Inception, DenseNet and AlexNet, but the pytorch implementation of the project was not able to finish because of lack of GPU memory. After debugging efforts, these issues need **major code refactoring**. This may be explained by the last update made by the authors on the pytorch version of the code which caused these bugs that will require more handling.


```
# Format:
$ python -m "knockoff.adversary.train" models/adversary/ADV_DIR ARCH DS_NAME \
        --budgets BUDGET1,BUDGET2,.. -d DEV_ID --pretrained --epochs EPOCHS \
        --lr LR
# DS_NAME refers to the dataset used to train victim model; used only to evaluate on test set during training of knockoff


# We run this command
$ python3 -m "knockoff.adversary.train" models/adversary/cubs200-resnet18-random \
        resnet34 CUBS200 --budgets 80000 -d 0 --pretrained imagenet \
        --log-interval 100 --epochs 10 --lr 0.01 
```




